In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import time
import traceback
import re
from datetime import datetime, time, timedelta
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
input_file_path = '/Users/sauravyadav/Documents/Repos/Datasets/BAR_EMA/output_data/data_draft_v2.csv'
output_file_path = '/Users/sauravyadav/Documents/Repos/Datasets/BAR_EMA/output_data/data_draft_v3.csv'

In [3]:
df = pd.read_csv(input_file_path)
df.head(2)

,SubID,Burst,Weekend,Day,RespID,RespType,UserID,SurvName,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,MR_wake,MR_rise,MR_bed,MR_FlSlp,MR_SlpQlt,MR_DrnkNum,MR_DrnkDur,MR_rlx,MR_hngvr,MR_Embrssd,MR_agrsv,MR_vom,MR_hrt,MR_rmbr,MR_work,MR_rude,MR_soc,MR_exprs,MR_guilt,MR_expctd,MR_mood,MR_mrng,MR_prmsd,MR_more,MR_fght,MR_enrgtc,MR_PsdOut,MR_MsdClss,MR_Impls,MR_sleep,MR_weed,MR_plan,chr,anoy,upbt,sad,irit,hap,bor,cnt,RP_enrgz,RP_excit,RP_sedat,RP_slotht,RP_slug,RP_up,RP_crntloc,RP_othrloc,RP_wthothr,RP_NumOth,RP_OthTyp,RP_OthWho,RP_EnjInt,RP_ActCld,RP_ActAcc,RP_OthCld,RP_OthAcc,RP_Vrtl,RP_AlnLke,RP_AlnGd,RP_WshOth,RP_lnly,RP_rjct,RP_incl,RP_acpt,RP_vrtint,RP_vrtoth,RP_vrtwho,RP_vrtenj,RP_vrtcld,RP_VrtAcc,RP_VrtOthCld,RP_VrtOthAcc,RP_NotInt,RP_WshInt,RP_lonely,RP_reject,RP_included,RP_accepted,RP_said,RP_spent,RP_impt,RP_spur,RP_Drnk
0,2001,1,1,Thu,5ab425aa2c9b2f5baaa69eaf,Submission,5a8381835893ca7070656612,Morning Reports,Assessment,2018-03-22,17:49:33,2018-03-22,17:52:41,2018-03-22,05:00:00,07:30:00,07:40:00,12:00:00,1.0,4.0,1.0,CONDITION_SKIPPED,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,71.0,27.0,79.0,21.0,9.0,69.0,11.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,1,1,Fri,5ab528815831424890f49c8c,Submission,5a8381835893ca7070656612,Daytime Surveys,Assessment,2018-03-23,12:15:03,2018-03-23,12:17:04,2018-03-23,12:14:59,07:15:00,07:18:00,11:30:00,2.0,4.0,3.0,1 hours 20 minutes,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,68.0,14.0,76.0,15.0,15.0,82.0,33.0,78.0,78.0,55.0,14.0,14.0,27.0,84.0,6.0,CONDITION_SKIPPED,1.0,11,"4,5,6,7",CONDITION_SKIPPED,59,41,42,58,35,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,0.0,0.0,33.0,67.0,2.0


### TO DO
- Add AM/PM information to MR time columns: 'MR_wake', 'MR_rise', 'MR_bed'

In [4]:
time_columns = ['MR_wake', 'MR_rise', 'MR_bed']
dur_cols = ['bed_wake', 'bed_rise', 'wake_rise']

for dur_col in dur_cols:
    df[f'MR_duration_{dur_col}'] = ""
    df[f'MR_duration_{dur_col}_minutes'] = None
    
# for time_col in time_columns:
#     df[time_col+'+'] = pd.to_datetime(df[time_col], format='%H:%M:%S')

df['MR_wake+'] = ""
df['MR_rise+'] = ""
df['MR_bed+'] = ""
# df['flag'] = 0

In [5]:


for index, row in tqdm(df.iterrows(), total=len(df)):

    if pd.isna(row.MR_wake) or pd.isna(row.MR_rise) or pd.isna(row.MR_bed):
        continue
        
    MR_wake = datetime.strptime(row.MR_wake, "%H:%M:%S")
    MR_rise = datetime.strptime(row.MR_rise, "%H:%M:%S")
    MR_bed = datetime.strptime(row.MR_bed, "%H:%M:%S")
    
    if not (MR_bed.hour > 12 or MR_rise.hour > 12 or MR_wake.hour > 12):
        
        # if MR_bed.hour in [4,5,6]:
        #     df.at[index, 'flag'] = 1
            
        # bedtime 7 8 9 10 11 12 13 14 15 16 assumption 7 pm to 4 am is bedtime
        if MR_bed.hour >= 7:
            MR_bed = MR_bed - timedelta(hours=12)
    
        
        # if MR_bed.hour in [12, 1, 2, 3, 4]:
        #     df.at[index, 'flag'] = 1
        
        # Assumption: Wake up time is 4 5 6 7 8 9 10 11 12 1 2 3 (where 1 2 and 3 are pm and rest am)
        if MR_wake.hour <= 2:
            MR_wake = MR_wake + timedelta(hours=12)
    
        if MR_rise.hour < MR_wake.hour:
            MR_rise = MR_rise + timedelta(hours=12)

    # print(f'{MR_rise.hour} : {MR_rise.minute} ')
    
    # # Assumption: Rise time is 7 8 9 10 11 12 1 2 3 (where 1 2 and 3 are pm and rest am)
    # if MR_rise.hour <= 3 or (MR_rise.hour == 4 and MR_rise.minute <=30):
    #     MR_rise = MR_rise + timedelta(hours=12)

    df.at[index, 'MR_wake+'] = MR_wake.strftime('%H:%M:%S')
    df.at[index, 'MR_rise+'] = MR_rise.strftime('%H:%M:%S')
    df.at[index, 'MR_bed+'] = MR_bed.strftime('%H:%M:%S')

    try:
        
        def diff(time_difference, col_name):
            hours, remainder = divmod(time_difference.seconds, 3600)
            minutes, seconds = divmod(remainder, 60)
            df.at[index, f'MR_duration_{col_name}'] = f"{hours} hours, {minutes} minutes"
            df.at[index, f'MR_duration_{col_name}_minutes'] = ((hours*60) + minutes)
            
        diff(abs(MR_bed - MR_wake), "bed_wake")
        diff(abs(MR_bed - MR_rise), "bed_rise")
        diff(abs(MR_wake - MR_rise), "wake_rise")
    
    except Exception as e:
        
        print(f"{e}: {MR_wake} {MR_rise} {MR_bed} {row.MR_wake}")
    
    # break
        
    
    

100%|██████████████████████████████████| 29981/29981 [00:02<00:00, 14599.01it/s]


In [6]:
df.sample()

,SubID,Burst,Weekend,Day,RespID,RespType,UserID,SurvName,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,MR_wake,MR_rise,MR_bed,MR_FlSlp,MR_SlpQlt,MR_DrnkNum,MR_DrnkDur,MR_rlx,MR_hngvr,MR_Embrssd,MR_agrsv,MR_vom,MR_hrt,MR_rmbr,MR_work,MR_rude,MR_soc,MR_exprs,MR_guilt,MR_expctd,MR_mood,MR_mrng,MR_prmsd,MR_more,MR_fght,MR_enrgtc,MR_PsdOut,MR_MsdClss,MR_Impls,MR_sleep,MR_weed,MR_plan,chr,anoy,upbt,sad,irit,hap,bor,cnt,RP_enrgz,RP_excit,RP_sedat,RP_slotht,RP_slug,RP_up,RP_crntloc,RP_othrloc,RP_wthothr,RP_NumOth,RP_OthTyp,RP_OthWho,RP_EnjInt,RP_ActCld,RP_ActAcc,RP_OthCld,RP_OthAcc,RP_Vrtl,RP_AlnLke,RP_AlnGd,RP_WshOth,RP_lnly,RP_rjct,RP_incl,RP_acpt,RP_vrtint,RP_vrtoth,RP_vrtwho,RP_vrtenj,RP_vrtcld,RP_VrtAcc,RP_VrtOthCld,RP_VrtOthAcc,RP_NotInt,RP_WshInt,RP_lonely,RP_reject,RP_included,RP_accepted,RP_said,RP_spent,RP_impt,RP_spur,RP_Drnk,MR_duration_bed_wake,MR_duration_bed_wake_minutes,MR_duration_bed_rise,MR_duration_bed_rise_minutes,MR_duration_wake_rise,MR_duration_wake_rise_minutes,MR_wake+,MR_rise+,MR_bed+
29074,2762,3,5,Sat,64bc30e1e6115c9e6a0e29f6,Submission,62aa6bc6f419c97443f41295,Daytime Surveys,Assessment,2023-07-22,15:40:08,2023-07-22,15:41:20,2023-07-22,15:32:32,12:00:00,12:23:00,05:00:00,1.0,2.0,13.0,8 hours 0 minutes,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,62.0,0.0,77.0,0.0,0.0,69.0,0.0,47.0,61.0,62.0,0.0,0.0,0.0,61.0,8.0,movie,1.0,7,3,CONDITION_SKIPPED,100,77,47,90,48,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,0.0,0.0,0.0,0.0,2.0,"7 hours, 0 minutes",420,"7 hours, 23 minutes",443,"0 hours, 23 minutes",23,12:00:00,12:23:00,05:00:00


In [7]:
# temp = df[['Day', 'MR_wake', 'MR_rise', 'MR_bed', 'MR_wake+', 'MR_rise+', 'MR_bed+', 'MR_duration_wake_rise_minutes', 
#            'MR_duration_bed_rise_minutes', 'MR_duration_bed_wake_minutes', 'MR_duration_bed_wake',
#            'MR_duration_bed_rise','MR_duration_wake_rise', 'MR_DrnkNum', 'MR_DrnkDur', 'MR_SlpQlt']]
# temp = temp.sort_values(by=['MR_duration_wake_rise_minutes', 'MR_duration_bed_rise_minutes'], ascending = [False, True]).reset_index(drop=True)

# # temp = temp[temp['MR_duration_wake_rise_minutes']>=300]
# # temp.to_csv('/Users/sauravyadav/Documents/Repos/Datasets/BAR_EMA/output_data/Sample_Inconsistencies.csv', index=False)

In [8]:
# temp.drop_duplicates().sample(30)

In [9]:
df.shape

(29981, 108)

In [10]:
df.to_csv(output_file_path,index=False)